# Merge and Analysis: NYC Noise Complaints vs Liquor Licenses

This notebook loads cleaned datasets, groups by ZIP code, merges them, calculates correlation, and creates visualizations.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Load Cleaned Datasets

In [ ]:
# Define data paths
data_dir = Path('../data/clean')
noise_file = data_dir / '311_noise_cleaned.csv'
liquor_file = data_dir / 'liquor_cleaned.csv'

# Load cleaned datasets
print("Loading cleaned datasets...\n")

df_noise = pd.read_csv(noise_file)
print(f"Loaded 311 noise complaints: {len(df_noise)} records")
print(f"Columns: {df_noise.columns.tolist()}")

df_liquor = pd.read_csv(liquor_file)
print(f"\nLoaded liquor licenses: {len(df_liquor)} records")
print(f"Columns: {df_liquor.columns.tolist()}")

## 3. Group Data by ZIP Code

In [ ]:
# Count noise complaints per ZIP code
print("Grouping noise complaints by ZIP code...")
noise_by_zip = df_noise.groupby('zip_code').size().reset_index(name='noise_count')
print(f"Unique ZIP codes with noise complaints: {len(noise_by_zip)}")
print("\nTop 10 ZIP codes by noise complaints:")
print(noise_by_zip.sort_values('noise_count', ascending=False).head(10))

In [ ]:
# Count liquor licenses per ZIP code
print("\nGrouping liquor licenses by ZIP code...")
liquor_by_zip = df_liquor.groupby('zip_code').size().reset_index(name='liquor_count')
print(f"Unique ZIP codes with liquor licenses: {len(liquor_by_zip)}")
print("\nTop 10 ZIP codes by liquor licenses:")
print(liquor_by_zip.sort_values('liquor_count', ascending=False).head(10))

## 4. Merge Counts by ZIP Code

In [ ]:
# Merge noise and liquor counts on ZIP code
print("Merging datasets by ZIP code...\n")
merged_data = pd.merge(
    noise_by_zip,
    liquor_by_zip,
    on='zip_code',
    how='outer'
)

# Fill NaN values with 0 (ZIP codes with only one type of data)
merged_data['noise_count'] = merged_data['noise_count'].fillna(0).astype(int)
merged_data['liquor_count'] = merged_data['liquor_count'].fillna(0).astype(int)

print(f"Merged dataset: {len(merged_data)} ZIP codes")
print(f"ZIP codes with both noise complaints and liquor licenses: {((merged_data['noise_count'] > 0) & (merged_data['liquor_count'] > 0)).sum()}")
print("\nFirst 10 rows of merged data:")
print(merged_data.head(10))

In [ ]:
# Summary statistics
print("\nSummary Statistics:")
print(merged_data[['noise_count', 'liquor_count']].describe())

## 5. Calculate Correlation

In [ ]:
# Calculate correlation coefficient
# Filter to ZIP codes that have at least one of each (for meaningful correlation)
merged_active = merged_data[(merged_data['noise_count'] > 0) & (merged_data['liquor_count'] > 0)].copy()

if len(merged_active) > 1:
    # Pearson correlation
    pearson_corr, pearson_pval = stats.pearsonr(merged_active['liquor_count'], merged_active['noise_count'])
    
    # Spearman correlation (rank-based, more robust to outliers)
    spearman_corr, spearman_pval = stats.spearmanr(merged_active['liquor_count'], merged_active['noise_count'])
    
    print("="*80)
    print("CORRELATION ANALYSIS")
    print("="*80)
    print(f"\nZIP codes analyzed: {len(merged_active)}")
    print(f"\nPearson Correlation Coefficient: {pearson_corr:.4f}")
    print(f"P-value: {pearson_pval:.4e}")
    print(f"Statistically significant: {'Yes' if pearson_pval < 0.05 else 'No'}")
    
    print(f"\nSpearman Correlation Coefficient: {spearman_corr:.4f}")
    print(f"P-value: {spearman_pval:.4e}")
    print(f"Statistically significant: {'Yes' if spearman_pval < 0.05 else 'No'}")
    
    # Interpretation
    print("\nInterpretation:")
    if abs(pearson_corr) < 0.3:
        strength = "weak"
    elif abs(pearson_corr) < 0.7:
        strength = "moderate"
    else:
        strength = "strong"
    
    direction = "positive" if pearson_corr > 0 else "negative"
    print(f"There is a {strength} {direction} correlation between liquor license density")
    print(f"and noise complaints across ZIP codes in Brooklyn and Staten Island.")
else:
    print("Not enough data points for correlation analysis.")

## 6. Create Scatter Plot with Trendline

In [ ]:
# Create scatter plot
if len(merged_active) > 1:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Scatter plot
    ax.scatter(
        merged_active['liquor_count'],
        merged_active['noise_count'],
        alpha=0.6,
        s=100,
        edgecolors='black',
        linewidths=0.5
    )
    
    # Add trendline
    z = np.polyfit(merged_active['liquor_count'], merged_active['noise_count'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(
        merged_active['liquor_count'].min(),
        merged_active['liquor_count'].max(),
        100
    )
    ax.plot(x_line, p(x_line), "r--", linewidth=2, label=f'Trendline: y = {z[0]:.2f}x + {z[1]:.2f}')
    
    # Labels and title
    ax.set_xlabel('Number of Liquor Licenses', fontsize=12, fontweight='bold')
    ax.set_ylabel('Number of Noise Complaints', fontsize=12, fontweight='bold')
    ax.set_title(
        'Correlation Between Liquor Licenses and Noise Complaints\nBrooklyn & Staten Island (2024)',
        fontsize=14,
        fontweight='bold',
        pad=20
    )
    
    # Add correlation coefficient to plot
    textstr = f'Pearson r = {pearson_corr:.4f}\np-value = {pearson_pval:.4e}\nn = {len(merged_active)} ZIP codes'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(
        0.05, 0.95,
        textstr,
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=props
    )
    
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Save the plot
    output_dir = Path('../output')
    output_dir.mkdir(parents=True, exist_ok=True)
    plot_file = output_dir / 'liquor_vs_noise_scatter.png'
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"\nSaved plot to: {plot_file}")
    
    plt.show()
else:
    print("Not enough data points to create scatter plot.")

## 7. Additional Visualizations

In [ ]:
# Create a bar chart showing top 10 ZIP codes by both metrics
if len(merged_active) > 0:
    # Get top 10 ZIP codes by total activity (noise + liquor)
    merged_active['total_activity'] = merged_active['noise_count'] + merged_active['liquor_count']
    top_10 = merged_active.nlargest(10, 'total_activity')
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(top_10))
    width = 0.35
    
    ax.bar(x - width/2, top_10['noise_count'], width, label='Noise Complaints', alpha=0.8)
    ax.bar(x + width/2, top_10['liquor_count'], width, label='Liquor Licenses', alpha=0.8)
    
    ax.set_xlabel('ZIP Code', fontsize=12, fontweight='bold')
    ax.set_ylabel('Count', fontsize=12, fontweight='bold')
    ax.set_title('Top 10 ZIP Codes by Activity\n(Noise Complaints + Liquor Licenses)', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(top_10['zip_code'], rotation=45, ha='right')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    # Save the plot
    plot_file2 = output_dir / 'top_10_zip_codes.png'
    plt.savefig(plot_file2, dpi=300, bbox_inches='tight')
    print(f"Saved plot to: {plot_file2}")
    
    plt.show()

## 8. Save Merged Data

In [ ]:
# Save merged data to CSV
output_file = output_dir / 'merged_analysis.csv'
merged_data.to_csv(output_file, index=False)
print(f"\nSaved merged data to: {output_file}")
print(f"Total ZIP codes: {len(merged_data)}")
print(f"Columns: {merged_data.columns.tolist()}")

## 9. Summary Report

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY REPORT")
print("="*80)
print(f"\nAnalysis Region: Brooklyn & Staten Island")
print(f"Time Period: 2024")
print(f"\nData Summary:")
print(f"  Total ZIP codes analyzed: {len(merged_data)}")
print(f"  ZIP codes with noise complaints: {(merged_data['noise_count'] > 0).sum()}")
print(f"  ZIP codes with liquor licenses: {(merged_data['liquor_count'] > 0).sum()}")
print(f"  ZIP codes with both: {len(merged_active)}")

print(f"\nNoise Complaints:")
print(f"  Total: {merged_data['noise_count'].sum():,}")
print(f"  Average per ZIP: {merged_data['noise_count'].mean():.1f}")
print(f"  Median per ZIP: {merged_data['noise_count'].median():.1f}")
print(f"  Max in single ZIP: {merged_data['noise_count'].max():,}")

print(f"\nLiquor Licenses:")
print(f"  Total: {merged_data['liquor_count'].sum():,}")
print(f"  Average per ZIP: {merged_data['liquor_count'].mean():.1f}")
print(f"  Median per ZIP: {merged_data['liquor_count'].median():.1f}")
print(f"  Max in single ZIP: {merged_data['liquor_count'].max():,}")

if len(merged_active) > 1:
    print(f"\nCorrelation Analysis:")
    print(f"  Pearson correlation: {pearson_corr:.4f} (p={pearson_pval:.4e})")
    print(f"  Spearman correlation: {spearman_corr:.4f} (p={spearman_pval:.4e})")
    print(f"  Statistical significance: {'Yes' if pearson_pval < 0.05 else 'No'}")

print(f"\nOutput Files:")
print(f"  Merged data: {output_file}")
if len(merged_active) > 1:
    print(f"  Scatter plot: {plot_file}")
    print(f"  Top 10 ZIP codes: {plot_file2}")

print("\n" + "="*80)